In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.4-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('logReg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df = spark.read.format('libsvm').load('data/sample_libsvm_data.txt')

In [6]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [12]:
train_data, test_data = df.randomSplit([0.7, 0.3])

In [13]:
clf = LogisticRegression()
clf_model = clf.fit(train_data)

In [14]:
summary = clf_model.summary

In [15]:
summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [16]:
results = clf_model.evaluate(test_data)

In [17]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.2574998893944...|[0.99999999941388...|       0.0|
|  0.0|(692,[98,99,100,1...|[26.3160956652934...|[0.99999999999627...|       0.0|
|  0.0|(692,[100,101,102...|[8.44409879765962...|[0.99978487986983...|       0.0|
|  0.0|(692,[123,124,125...|[30.3393635769708...|[0.99999999999993...|       0.0|
|  0.0|(692,[123,124,125...|[19.7998420635247...|[0.99999999748210...|       0.0|
|  0.0|(692,[124,125,126...|[30.5152324401518...|[0.99999999999994...|       0.0|
|  0.0|(692,[124,125,126...|[29.2300843645496...|[0.99999999999979...|       0.0|
|  0.0|(692,[124,125,126...|[18.5700322914913...|[0.99999999138733...|       0.0|
|  0.0|(692,[124,125,126...|[23.0630390858764...|[0.99999999990365...|       0.0|
|  0.0|(692,[126

In [18]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [19]:
binary_evaluator = BinaryClassificationEvaluator()
eval_roc = binary_evaluator.evaluate(results.predictions)

In [20]:
eval_roc

1.0